In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from hyperopt import hp
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe
import mlflow
from mlflow.models import infer_signature
import os
from mlflow import MlflowClient
import git

## MLFLOW

In [2]:
mlflow.set_tracking_uri(uri="http://localhost:5000")

In [3]:


## load data

#On recupere les données depuis les differents csv puis on les stocks dans differentes variables correspondantes (jeu de train dans X_train)

df_train = pd.read_csv("../data/archive/train.csv")
df_train
X_train, y_train = df_train["review"],  df_train["polarity"]

df_test = pd.read_csv("../data/archive/test.csv")
X_test, y_test = df_train["review"],  df_train["polarity"]
df_val = pd.read_csv("../data/archive/valid.csv")
X_val, y_val= df_train["review"],  df_train["polarity"]

Nous verifions la présence de données manquantes

In [4]:
df_train.isna().sum(), df_train.isnull().sum()

(Unnamed: 0    0
 film-url      0
 review        0
 polarity      0
 dtype: int64,
 Unnamed: 0    0
 film-url      0
 review        0
 polarity      0
 dtype: int64)

verification de la balance des labels

In [5]:
np.unique(df_train["polarity"], return_counts=True)

(array([0, 1]), array([79413, 80587]))

it's balance

## preprocessing

In [6]:
STOP_WORDS = list(
    """
a à â abord afin ah ai aie ainsi ait allaient allons
alors anterieur anterieure anterieures antérieur antérieure antérieures
apres après as assez attendu au
aupres auquel aura auraient aurait auront
aussi autre autrement autres autrui aux auxquelles auxquels avaient
avais avait avant avec avoir avons ayant

bas basee bat

c' c’ ça car ce ceci cela celle celle-ci celle-la celle-là celles celles-ci celles-la celles-là
celui celui-ci celui-la celui-là cent cependant certain certaine certaines certains certes ces
cet cette ceux ceux-ci ceux-là chacun chacune chaque chez ci cinq cinquantaine cinquante
cinquantième cinquième combien comme comment compris concernant

d' d’ da dans de debout dedans dehors deja dejà delà depuis derriere
derrière des desormais desquelles desquels dessous dessus deux deuxième
deuxièmement devant devers devra different differente differentes differents différent
différente différentes différents dire directe directement dit dite dits divers
diverse diverses dix dix-huit dix-neuf dix-sept dixième doit doivent donc dont
douze douzième du duquel durant dès déja déjà désormais

effet egalement eh elle elle-meme elle-même elles elles-memes elles-mêmes en encore
enfin entre envers environ es ès est et etaient étaient etais étais etait était
etant étant etc etre être eu eux eux-mêmes exactement excepté également

fais faisaient faisant fait facon façon feront font

gens

ha hem hep hi ho hormis hors hou houp hue hui huit huitième
hé i il ils importe

j' j’ je jusqu jusque juste

l' l’ la laisser laquelle le lequel les lesquelles lesquels leur leurs longtemps
lors lorsque lui lui-meme lui-même là lès

m' m’ ma maint maintenant mais malgre malgré me meme memes merci mes mien
mienne miennes miens mille moi moi-meme moi-même moindres moins
mon même mêmes

n' n’ na ne neanmoins neuvième ni nombreuses nombreux nos notamment
notre nous nous-mêmes nouveau nul néanmoins nôtre nôtres

o ô on ont onze onzième or ou ouias ouste outre
ouvert ouverte ouverts où

par parce parfois parle parlent parler parmi partant
pas pendant pense permet personne peu peut peuvent peux plus
plusieurs plutot plutôt possible possibles pour pourquoi
pourrais pourrait pouvait prealable precisement
premier première premièrement
pres procedant proche près préalable précisement pu puis puisque

qu' qu’ quand quant quant-à-soi quarante quatorze quatre quatre-vingt
quatrième quatrièmement que quel quelconque quelle quelles quelqu'un quelque
quelques quels qui quiconque quinze quoi quoique

relative relativement rend rendre restant reste
restent retour revoici revoila revoilà

s' s’ sa sait sans sauf se seize selon semblable semblaient
semble semblent sent sept septième sera seraient serait seront ses seul seule
seulement seuls seules si sien sienne siennes siens sinon six sixième soi soi-meme soi-même soit
soixante son sont sous souvent specifique specifiques spécifique spécifiques stop
suffisant suffisante suffit suis suit suivant suivante
suivantes suivants suivre sur surtout

t' t’ ta tant te tel telle tellement telles tels tenant tend tenir tente
tes tien tienne tiennes tiens toi toi-meme toi-même ton touchant toujours tous
tout toute toutes treize trente tres trois troisième troisièmement très
tu té

un une unes uns

va vais vas vers via vingt voici voila voilà vont vos
votre votres vous vous-mêmes vu vé vôtre vôtres

y

""".split()
)

source https://github.com/explosion/spaCy/blob/master/spacy/lang/fr/stop_words.py


we use  TfidfVectorizer, it use count vectorizer followed bytransform tfid
- count vectorizer create a matrice document x word  the cell i,j correspond of ith document avec jth word and it count the  number of occurence of the word j in i
- tfid use this matrix to calculate the tfidf matrix

### regression lineaire

In [7]:
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 42,
}

In [8]:
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(**params))
    ])# creation de la pipeline avec le preprocessing et le modèle
    

In [9]:
session_name = os.environ.get('JPY_SESSION_NAME', '')
notebook_name = os.path.basename(session_name)
notebook_name

'model_design_2.ipynb'

In [10]:
path = os.getcwd()
repo = git.Repo(os.path.abspath(os.path.join(path, os.pardir)))
sha_commit = repo.head.object.hexsha 
branch = repo.active_branch.name

In [11]:
branch, sha_commit

('main', 'd0db016196ac6e6f64e255f3812b58b122e7312a')

In [13]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow LRregression")
# Start an MLflow run
with mlflow.start_run(description="pipeline lr + tfidf") as run:
    mlflow.sklearn.autolog(log_datasets=False)
    pipeline.fit(X_train, y_train)
    accuracy = pipeline.score(X_test, y_test)
    mlflow.log_metric("accuracy_test", accuracy)
    mlflow.set_tag("mlflow.source.name",notebook_name)
    mlflow.set_tag("mlflow.source.git.commit", sha_commit) 
    signature = infer_signature(X_train, pipeline.predict(X_train))
    mlflow.set_tag("mlflow.source.git.branch", branch)
    mlflow.sklearn.log_model(pipeline, artifact_path="sklearn-model", signature=signature)


# Create source model version
client = MlflowClient()
src_name = "LR-staging"
client.create_registered_model(src_name)
src_uri = f"runs:/{run.info.run_id}/sklearn-model"
mv_src = client.create_model_version(src_name, src_uri, run.info.run_id)


/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
2024/11/12 17:48:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run whimsical-shoat-810 at: http://localhost:5000/#/experiments/257713885670805413/runs/149095e72f714103987031079705775c.
2024/11/12 17:48:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/257713885670805413.
2024/11/12 17:48:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR-staging, version 1


Consultez les métriques logguées par autolog lors de l'entrainement. Qu'en pensez-vous ? Ces métriques sont-elles satisfaisantes pour évaluer la qualité de votre modèle?

Ajoutez au moins une métrique pertinente d'évaluation de votre modèle.
c'est l'accuracy du training on preferait l'avoir sur le test

## hyperopt

In [14]:
#  optimisation des hyperparametres  pour la regression logistic
def objective(params):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=params['tfidf__max_features'])),
        ('logreg', LogisticRegression(C=params['logreg__C']))
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    
    return {'loss': 1 - accuracy, 'status': "ok"}

space = {
    'tfidf__max_features': hp.choice('tfidf__max_features', [1000, 5000, 10000]),
    'logreg__C': hp.loguniform('logreg__C', np.log(0.001), np.log(100))
}

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,
    return_argmin=False
)


  0%|                                                        | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/11/12 17:48:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd9364ee866b74a51a1c54abc0ace3c45', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/11/12 17:49:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run able-penguin-876 at: http://localhost:5000/#/experiments/257713885670805413/runs/d9364ee866b74a51a1c54abc0ace3c45.

2024/11/12 17:49:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/257713885670805413.



 10%|██▉                          | 1/10 [00:48<07:15, 48.38s/trial, best loss: 0.11268124999999996]

2024/11/12 17:49:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f3511e91eb7148569d2e268a4455143b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/11/12 17:50:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run abrasive-shrew-904 at: http://localhost:5000/#/experiments/257713885670805413/runs/f3511e91eb7148569d2e268a4455143b.

2024/11/12 17:50:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/257713885670805413.



 20%|█████▊                       | 2/10 [01:37<06:31, 48.89s/trial, best loss: 0.08045000000000002]

...

 80%|███████████████████████▏     | 8/10 [06:34<01:38, 49.11s/trial, best loss: 0.05758125000000003]

2024/11/12 17:55:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd9bf506fd3004f6f8a54128cdc171367', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/11/12 17:55:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run rare-koi-141 at: http://localhost:5000/#/experiments/257713885670805413/runs/d9bf506fd3004f6f8a54128cdc171367.

2024/11/12 17:55:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/257713885670805413.



 90%|██████████████████████████   | 9/10 [07:24<00:49, 49.17s/trial, best loss: 0.05758125000000003]

2024/11/12 17:55:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ed63cb5d6b17448bb7d4df1676a222c5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/11/12 17:56:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run skillful-wolf-580 at: http://localhost:5000/#/experiments/257713885670805413/runs/ed63cb5d6b17448bb7d4df1676a222c5.

2024/11/12 17:56:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/257713885670805413.



100%|████████████████████████████| 10/10 [08:11<00:00, 49.20s/trial, best loss: 0.05758125000000003]


In [16]:
best

{'logreg__C': 40.7430245196735, 'tfidf__max_features': 10000}